## 一、配置环境变量

In [16]:
import os
from dotenv import dotenv_values

In [31]:
env_variables = dotenv_values('.env')
for var in env_variables:
    print(var)
    os.environ[var] = env_variables[var]

ZHIPUAI_API_KEY
DASHSCOPE_API_KEY


In [32]:
os.getenv("DASHSCOPE_API_KEY")

'sk-feddf4791axxxxxxx21f8cc335e83'

## 二、配置LLM

In [19]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os
from typing import Dict, List, Optional, Tuple, Union

from zhipuai import ZhipuAI
import dashscope
import random
from http import HTTPStatus
import dashscope
from dashscope import Generation  # 建议dashscope SDK 的版本 >= 1.14.0

PROMPT_TEMPLATE = dict(
    RAG_PROMPT_TEMPALTE="""使用以上下文来回答用户的问题。如果你不知道答案，就说你不知道。总是使用中文回答。
        问题: {question}
        可参考的上下文：
        ···
        {context}
        ···
        如果给定的上下文无法让你做出回答，请回答数据库中没有这个内容，你不知道。
        有用的回答:""",
    InternLM_PROMPT_TEMPALTE="""先对上下文进行内容总结,再使用上下文来回答用户的问题。如果你不知道答案，就说你不知道。总是使用中文回答。
        问题: {question}
        可参考的上下文：
        ···
        {context}
        ···
        如果给定的上下文无法让你做出回答，请回答数据库中没有这个内容，你不知道。
        有用的回答:"""
)


class BaseModel:
    def __init__(self, path: str = '') -> None:
        self.path = path

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        pass

    def load_model(self):
        pass

class GLM4Chat(BaseModel):
    def __init__(self, path: str = '', model: str = "glm-4") -> None:
        super().__init__(path)
        self.model = model

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        client = ZhipuAI(api_key=os.getenv("ZHIPUAI_API_KEY"))  # 填写您自己的APIKey
        history.append({'role': 'user', 'content': PROMPT_TEMPLATE['RAG_PROMPT_TEMPALTE'].format(question=prompt, context=content)})
        response = client.chat.completions.create(
            model="glm-4",  # 填写需要调用的模型名称
            messages=history
        )
        return response.choices[0].message

class QwenChat(BaseModel):
    def __init__(self, path: str = '', model: str = "qwen-turbo") -> None:
        super().__init__(path)
        dashscope.api_key = os.getenv("DASHSCOPE_API_KEY")
        self.model = model

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        history.append({'role': 'user', 'content': PROMPT_TEMPLATE['RAG_PROMPT_TEMPALTE'].format(question=prompt, context=content)})
        response = Generation.call(model="qwen-turbo",
                                messages=history,
                                # 设置随机数种子seed，如果没有设置，则随机数种子默认为1234
                                seed=random.randint(1, 10000),
                                # 将输出设置为"message"格式
                                result_format='message')
        if response.status_code == HTTPStatus.OK:
            return response.output.choices[0].message["content"]
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))


class OpenAIChat(BaseModel):
    def __init__(self, path: str = '', model: str = "gpt-3.5-turbo-1106") -> None:
        super().__init__(path)
        self.model = model

    def chat(self, prompt: str, history: List[dict], content: str) -> str:
        from openai import OpenAI
        client = OpenAI()   
        history.append({'role': 'user', 'content': PROMPT_TEMPLATE['RAG_PROMPT_TEMPALTE'].format(question=prompt, context=content)})
        response = client.chat.completions.create(
            model=self.model,
            messages=history,
            max_tokens=150,
            temperature=0.1
        )
        return response.choices[0].message.content

class InternLMChat(BaseModel):
    def __init__(self, path: str = '') -> None:
        super().__init__(path)
        self.load_model()

    def chat(self, prompt: str, history: List = [], content: str='') -> str:
        prompt = PROMPT_TEMPLATE['InternLM_PROMPT_TEMPALTE'].format(question=prompt, context=content)
        response, history = self.model.chat(self.tokenizer, prompt, history)
        return response


    def load_model(self):
        import torch
        from transformers import AutoTokenizer, AutoModelForCausalLM
        self.tokenizer = AutoTokenizer.from_pretrained(self.path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(self.path, torch_dtype=torch.float16, trust_remote_code=True).cuda()


In [20]:
qwenchat = QwenChat()
qwenchat.chat(history=[], prompt="hello", content="")

'你好！有什么我可以帮助你的吗？'

## 三、设置最基础的向量数据库VectorStore

In [21]:
import os
import time
from typing import Dict, List, Optional, Tuple, Union
import json
from RAG.Embeddings import BaseEmbeddings, OpenAIEmbedding, JinaEmbedding, ZhipuEmbedding
import numpy as np
from tqdm import tqdm


class VectorStore():
    def __init__(self, document: List[str] = ['']) -> None:
        self.document = document

    def get_vector(self, EmbeddingModel: BaseEmbeddings) -> List[List[float]]:
        
        self.vectors = []
        for doc in tqdm(self.document, desc="Calculating embeddings"):
            self.vectors.append(EmbeddingModel.get_embedding(doc))
        return self.vectors

    def batch_split_list(self, lst,batch_size):
        return [lst[i:i + batch_size] for i in range(0, len(lst), batch_size)]

    def get_vector_batch(self, EmbeddingModel: BaseEmbeddings, batch:int) -> List[List[float]]:

        self.vectors = []
        self.document = self.batch_split_list(self.document, batch)
        for doc in tqdm(self.document, desc="Calculating embeddings"):
            self.vectors.extend(EmbeddingModel.get_embeddings(doc))
        return self.vectors

    def persist(self, path: str = 'storage'):
        if not os.path.exists(path):
            os.makedirs(path)
        with open(f"{path}/doecment.json", 'w', encoding='utf-8') as f:
            json.dump(self.document, f, ensure_ascii=False)
        if self.vectors:
            with open(f"{path}/vectors.json", 'w', encoding='utf-8') as f:
                json.dump(self.vectors, f)

    def load_vector(self, path: str = 'storage'):
        with open(f"{path}/vectors.json", 'r', encoding='utf-8') as f:
            self.vectors = json.load(f)
        with open(f"{path}/doecment.json", 'r', encoding='utf-8') as f:
            self.document = json.load(f)

    def get_similarity(self, vector1: List[float], vector2: List[float]) -> float:
        return BaseEmbeddings.cosine_similarity(vector1, vector2)

    def query(self, query: str, EmbeddingModel: BaseEmbeddings, k: int = 1) -> List[str]:
        query_vector = EmbeddingModel.get_embedding(query)

        end_time = time.time()

        result = np.array([self.get_similarity(query_vector, vector)
                          for vector in self.vectors])
        print(' 检索 cost %f second' % (time.time() - end_time))
        return np.array(self.document)[result.argsort()[-k:][::-1]].tolist()

In [22]:
vector = VectorStore()

In [23]:
vector.load_vector('../../storage/github_data') # 加载本地的知识库

In [24]:
len(vector.vectors[0])

1024

## 四、定义embedding算法
NOTICE：
1.需要注意的是PaddleEmbedding()类中向量维度为768，而ZhipuEmbedding()类中向量维度为1024


In [25]:
from RAG.PaddleEmbedding import PaddleEmbedding
embedding = ZhipuEmbedding()# 创建EmbeddingModel

In [26]:
question = 'Git中的文件有哪几种状态?'
content = vector.query(question, EmbeddingModel=embedding, k=2)[0]
print(f'知识库输出：{content}')

 检索 cost 0.005692 second
知识库输出：包含了实际的文件，而暂存区是一个准备好下次提交的文件列表。
提交更改
提交是Git中的基本操作，它会将暂存区的更改记录到仓库中。每次提交都会在仓库中创建一个快照，并允许之后恢复到该状态。
bash
gitcommit-m"描述性的提交信息"
查看提交历史
要查看提交历史，可以使用gitlog命令：
gitlog
还可以使用多种选项来定制显示的日志输出。
撤销操作
如果需要撤销操作，Git提供了几个命令：

撤销工作区的修改：

bash
gitcheckout--<文件名>

撤销暂存区的文件：

bash
gitresetHEAD<文件名>

撤销提交（创建一个新的提交来撤销之前的提交）：

bash
gitrevert<提交ID>
标签管理
标签是指向特定提交的引用，通常用于版本发布。创建一个新标签：
bash
gittag<标签名>
列出所有标签：
bash
gittag
删除标签：
bash
gittag-d<标签名>
查看标签信息：
bash
gitshow<标签名>
推送标签到远程仓库：
bash
gitpushorigin<标签名>
分支管理
分支的概念
在Git中，分支是用来隔离开发工作的。每个分支都是一个独立的开发环境，互不影响。分支可以很方便地被创建和合并，因此许多开发者使用分支来进行特性开发、修复bug或者尝试新想法。
Git的一个核心概念是几乎所有操作都是本地执行的，分支也不例外。这意味着你在本地创建或切换分支，不需要与远程仓库进行通信。
创建与合并分支
在Git中创建新分支可以使用gitbranch命令，合并分支则使用gitmerge命令。
```bash
创建新分支



In [27]:
chat = QwenChat()
print(f"知识库输出：{chat.chat(question, [], content)}")

知识库输出：在Git中，文件主要有三种状态：

1. 工作区（Working Directory）：这是你的开发环境，包含了你当前正在编辑的文件。当你从仓库检出代码或者直接修改文件时，这些修改都在工作区。

2. 暂存区（Staging Area, Index）：也称为索引，是一个缓冲区，当你对工作区的文件做了修改后，可以选择哪些修改添加到暂存区，准备提交。暂存区的文件是未提交但已准备好的更改。

3. 仓库（Repository）：Git的中央存储库，包含了所有的提交历史、分支和标签。提交后的文件状态会保存在仓库中，可以通过提交ID访问。

当提到Git中的文件状态时，通常指的是这三种状态。工作区和暂存区是本地的状态，仓库则是版本控制的状态。
